In [0]:
%pip install openpyxl --upgrade
dbutils.library.restartPython()

In [0]:
import pandas as pd

# Load dataset from UCI public URL
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
df_pd = pd.read_excel(url, engine='openpyxl', dtype=str)

# Convert to Spark DataFrame
df_spark = spark.createDataFrame(df_pd)

# Save to Bronze Zone in ADLS
#df_spark.write.format("delta").mode("overwrite").save("abfss://bronze@straccountretail.dfs.core.windows.net/online_retail")
df_spark.write.format("delta").mode("overwrite").save("abfss://bronze@rgdbronzestretail0001.dfs.core.windows.net/online_retail")

